# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-17-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-17-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-18 04:27:56,33.939110,67.709953,37599,1375,27166,9058.0,Afghanistan,96.585159,3.657012
1,NaN,NaN,NaN,Albania,2020-08-18 04:27:56,41.153300,20.168300,7499,230,3816,3453.0,Albania,260.580999,3.067076
2,NaN,NaN,NaN,Algeria,2020-08-18 04:27:56,28.033900,1.659600,39025,1379,27347,10299.0,Algeria,88.994462,3.533632
3,NaN,NaN,NaN,Andorra,2020-08-18 04:27:56,42.506300,1.521800,1005,53,869,83.0,Andorra,1300.718307,5.273632
4,NaN,NaN,NaN,Angola,2020-08-18 04:27:56,-11.202700,17.873900,1935,88,632,1215.0,Angola,5.887495,4.547804
5,NaN,NaN,NaN,Antigua and Barbuda,2020-08-18 04:27:56,17.060800,-61.796400,93,3,88,2.0,Antigua and Barbuda,94.967731,3.225806
6,NaN,NaN,NaN,Argentina,2020-08-18 04:27:56,-38.416100,-63.616700,299126,5814,217850,75462.0,Argentina,661.845021,1.943663
7,NaN,NaN,NaN,Armenia,2020-08-18 04:27:56,40.069100,45.038200,41701,824,34655,6222.0,Armenia,1407.280019,1.975972
8,NaN,NaN,Australian Capital Territory,Australia,2020-08-18 04:27:56,-35.473500,149.012400,113,3,110,0.0,"Australian Capital Territory, Australia",26.395702,2.654867
9,NaN,NaN,New South Wales,Australia,2020-08-18 04:27:56,-33.868800,151.209300,3959,52,2994,913.0,"New South Wales, Australia",48.768169,1.313463


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37054,37054,37162,37269,37345,37424,37431,37551,37596,37599
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6275,6411,6536,6676,6817,6971,7117,7260,7380,7499
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,34693,35160,35712,36204,36699,37187,37664,38133,38583,39025


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1312,1312,1328,1344,1354,1363,1363,1370,1375,1375
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,193,199,200,205,208,213,219,225,228,230
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1293,1302,1312,1322,1333,1341,1351,1360,1370,1379


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25960,25960,26228,26415,26694,26714,26714,27166,27166,27166
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3268,3342,3379,3480,3552,3616,3695,3746,3794,3816
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,24083,24506,24920,25263,25627,26004,26308,26644,27017,27347


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1086,1169,1174,1162,1188,1196,1196,1206,1218,1219
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3502,3544,3651,3670,3710,3744,3791,3839,3877,3888
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,573,575,576,579,581,586,590,598,604,605


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,21,21,21,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,24,25,25,29,29,29,29,29,29,29
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5,5,5,6,6,6,6,6,6,6


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-18 04:27:56,32.539527,-86.644082,1219,22,0,1197.0,"Autauga, Alabama, US",2181.889778,1.804758
668,1075.0,Lamar,Alabama,US,2020-08-18 04:27:56,33.779950,-88.096680,245,2,0,243.0,"Lamar, Alabama, US",1774.719305,0.816327
669,1077.0,Lauderdale,Alabama,US,2020-08-18 04:27:56,34.901719,-87.656247,1240,21,0,1219.0,"Lauderdale, Alabama, US",1337.229993,1.693548


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5438325,170497,1865580
Brazil,3359570,108536,2699080
India,2702681,51797,1977671


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5438325,170497,1865580,3402248,2020-08-18 04:27:56,37.914208,-91.353830
Brazil,3359570,108536,2699080,551954,2020-08-18 04:27:56,-12.669522,-48.480493
India,2702681,51797,1977671,673213,2020-08-18 04:27:56,23.088275,81.806127
Russia,925558,15707,734573,175278,2020-08-18 04:27:56,54.546312,62.120860
South Africa,589886,11982,477671,100233,2020-08-18 04:27:56,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,629415,11296,0
Florida,576094,9539,0
Texas,561110,10447,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,629415,11296,0,618119,2020-08-18 04:27:56,37.843962,-120.728594
Florida,576094,9539,0,566555,2020-08-18 04:27:56,28.940755,-82.700744
Texas,561110,10447,0,550663,2020-08-18 04:27:56,31.660643,-98.653069
New York,425916,32846,0,393070,2020-08-18 04:27:56,42.671593,-75.579694
Georgia,238861,4727,0,234134,2020-08-18 04:27:56,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,230458,23634,0
California,Los Angeles,223230,5273,0
Florida,Miami-Dade,146026,2081,0
Arizona,Maricopa,129562,2596,0
Illinois,Cook,116620,4964,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,230458,23634,0,206824,2020-08-18 04:27:56,40.767273,-73.971526,36061.0
California,Los Angeles,223230,5273,0,217957,2020-08-18 04:27:56,34.308284,-118.228241,6037.0
Florida,Miami-Dade,146026,2081,0,143945,2020-08-18 04:27:56,25.611236,-80.551706,12086.0
Arizona,Maricopa,129562,2596,0,126966,2020-08-18 04:27:56,33.348359,-112.491815,4013.0
Illinois,Cook,116620,4964,0,111656,2020-08-18 04:27:56,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,37551,7260,38133,989,1879,93,289100,41495,23287,23179,...,319232,1434,34528,32607,951,16153,10,1858,9186,5176
2020-08-16,37596,7380,38583,989,1906,93,294569,41663,23558,23370,...,320343,1440,35329,33755,964,16534,10,1869,9343,5261
2020-08-17,37599,7499,39025,1005,1935,93,299126,41701,23773,23534,...,321064,1457,35702,34802,983,16844,10,1882,9839,5308


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,1370,225,1360,53,86,3,5637,817,396,728,...,41446,38,225,276,24,108,1,528,260,130
2020-08-16,1375,228,1370,53,88,3,5703,818,421,728,...,41451,38,232,281,24,110,1,530,260,132
2020-08-17,1375,230,1379,53,88,3,5814,824,438,729,...,41454,40,236,288,25,113,1,535,264,135


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,27166,3746,26644,863,628,83,205697,34484,13633,20627,...,1486,1194,29328,21747,447,9388,8,1013,8065,2047
2020-08-16,27166,3794,27017,863,628,88,211702,34584,14083,20681,...,1486,1200,30973,22700,456,9838,8,1013,8412,2092
2020-08-17,27166,3816,27347,869,632,88,217850,34655,14536,20765,...,1490,1205,30973,23575,467,9906,8,1045,8575,3848


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1086,1169,1174,1162,1188,1196,1196,1206,1218,1219
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,220,222,229,230,230,237,237,239,240,245
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1157,1167,1189,1190,1203,1213,1219,1231,1238,1240
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,348,349,351,354,356,360,364,367,370,370
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2660,2678,2700,2708,2724,2736,2735,2760,2776,2779


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-15,107580,4156,0,192654,51992,617994,52819,50897,16396,49,...,131747,546552,46321,1509,734,105750,66885,8462,65056,3227
2020-08-16,108433,4259,0,193537,52665,623873,53157,50897,16451,49,...,133708,555394,46652,1515,741,106687,67461,8567,65741,3286
2020-08-17,109004,4309,0,194005,53077,629415,53351,51267,16536,49,...,134744,561110,46894,1527,760,107421,67721,8641,66196,3331


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-15        1206    3839     598  469    854     495    788    1903   
2020-08-16        1218    3877     604  469    860     496    792    1919   
2020-08-17        1219    3888     605  474    863     496    795    1936   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-15          854      299  ...     144      6       81       40   
2020-08-16          854      302  ...     144      6       83       40   
2020-08-17          855      306  ...     146      6       86       40   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-15            275   385   276          0       97      7  
2020-08-16            276   389   276          0       97      7  
2020-08-17            278   390   277          0      100      8  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-15,1896,28,0,4492,600,11218,1896,4453,593,0,...,1345,10268,363,58,9,2381,1766,160,1038,30
2020-08-16,1898,28,0,4506,599,11243,1896,4453,593,0,...,1366,10396,363,58,9,2381,1781,160,1039,30
2020-08-17,1925,28,0,4506,603,11296,1896,4456,593,0,...,1387,10447,364,58,9,2385,1785,160,1039,30


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-15          22      29       6    5      5      11     36      19   
2020-08-16          22      29       6    5      5      11     36      19   
2020-08-17          22      29       6    5      5      13     36      21   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-15           38        8  ...       0      0        0        0   
2020-08-16           38        8  ...       0      0        0        0   
2020-08-17           38        8  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-15              0     0     0         29        0      0  
2020-08-16              0     0     0         29        0      0  
2020-08-17              0     0     0         29        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,0.003206,0.020093,0.012452,0.000000,0.014579,0.0,0.023591,0.004746,0.010940,0.013245,...,0.003275,0.009148,0.020904,0.039068,0.022581,0.020147,0.0,0.000000,0.018291,0.020505
2020-08-16,0.001198,0.016529,0.011801,0.000000,0.014369,0.0,0.018917,0.004049,0.011637,0.008240,...,0.003480,0.004184,0.023199,0.035207,0.013670,0.023587,0.0,0.005920,0.017091,0.016422
2020-08-17,0.000080,0.016125,0.011456,0.016178,0.015215,0.0,0.015470,0.000912,0.009126,0.007018,...,0.002251,0.011806,0.010558,0.031018,0.019710,0.018749,0.0,0.006956,0.053088,0.008934


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,0.005136,0.027397,0.006662,0.0,0.000000,0.0,0.019902,0.003686,0.044855,0.004138,...,0.000072,0.000000,0.022727,0.037594,0.090909,0.018868,0.0,0.000000,0.015625,0.015625
2020-08-16,0.003650,0.013333,0.007353,0.0,0.023256,0.0,0.011708,0.001224,0.063131,0.000000,...,0.000121,0.000000,0.031111,0.018116,0.000000,0.018519,0.0,0.003788,0.000000,0.015385
2020-08-17,0.000000,0.008772,0.006569,0.0,0.000000,0.0,0.019463,0.007335,0.040380,0.001374,...,0.000072,0.052632,0.017241,0.024911,0.041667,0.027273,0.0,0.009434,0.015385,0.022727


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,0.01692,0.013802,0.012772,0.000000,0.075342,0.000000,0.033627,0.009367,0.000073,0.006244,...,0.003376,0.010152,0.054016,0.007739,0.022883,0.000640,0.0,0.003964,0.063143,0.024525
2020-08-16,0.00000,0.012814,0.013999,0.000000,0.000000,0.060241,0.029193,0.002900,0.033008,0.002618,...,0.000000,0.005025,0.056090,0.043822,0.020134,0.047934,0.0,0.000000,0.043025,0.021983
2020-08-17,0.00000,0.005799,0.012215,0.006952,0.006369,0.000000,0.029041,0.002053,0.032166,0.004062,...,0.002692,0.004167,0.000000,0.038546,0.024123,0.006912,0.0,0.031589,0.019377,0.839388


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-15,0.011956,0.020378,NaN,0.004866,-0.007635,0.007981,0.005712,0.00000,0.003427,0.0,...,0.009881,0.012747,0.007504,0.005330,0.042614,0.008699,0.011279,0.022351,0.012907,0.013823
2020-08-16,0.007929,0.024783,NaN,0.004583,0.012944,0.009513,0.006399,0.00000,0.003354,0.0,...,0.014885,0.016178,0.007146,0.003976,0.009537,0.008861,0.008612,0.012408,0.010529,0.018283
2020-08-17,0.005266,0.011740,NaN,0.002418,0.007823,0.008883,0.003650,0.00727,0.005167,0.0,...,0.007748,0.010292,0.005187,0.007921,0.025641,0.006880,0.003854,0.008638,0.006921,0.013694


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-15      0.008361  0.012662  0.013559  0.008602  0.007075  0.002024   
2020-08-16      0.009950  0.009898  0.010033  0.000000  0.007026  0.002020   
2020-08-17      0.000821  0.002837  0.001656  0.010661  0.003488  0.000000   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-15      0.011553  0.019282  0.002347  0.020478  ...  0.000000    0.0   
2020-08-16      0.005076  0.008408  0.000000  0.010033  ...  0.000000    0.0   
2020-08-17      0.003788  0.008859  0.001171  0.013245  ...  0.013889    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-15      0.038462      0.0   0.014760  0.005222 -0.003610        NaN   
2020-08-16      0.024691      0.0   0.003636  0.010390  0.000000        NaN   
2020-08-17      0.036145      0.0   0.007246  0.002571  0.003623        NaN   

Province_State                      
Admin2          Washakie    Weston  
2020-08-15      0.010417  0.400000  
2020-08-16      0.000000  0.000000  
2020-08-17      0.030928  0.142857  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-15,0.001585,0.037037,NaN,0.015600,0.022147,0.006279,0.004237,0.000000,0.0,NaN,...,0.014329,0.018853,0.008333,0.0,0.0,0.004641,0.006268,0.019108,0.012683,0.0
2020-08-16,0.001055,0.000000,NaN,0.003117,-0.001667,0.002229,0.000000,0.000000,0.0,NaN,...,0.015613,0.012466,0.000000,0.0,0.0,0.000000,0.008494,0.000000,0.000963,0.0
2020-08-17,0.014226,0.000000,NaN,0.000000,0.006678,0.004714,0.000000,0.000674,0.0,NaN,...,0.015373,0.004906,0.002755,0.0,0.0,0.001680,0.002246,0.000000,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga Baldwin Barbour Bibb Blount   Bullock Butler   Calhoun   
2020-08-15         0.0     0.0     0.0  0.0    0.0  0.000000    0.0  0.000000   
2020-08-16         0.0     0.0     0.0  0.0    0.0  0.000000    0.0  0.000000   
2020-08-17         0.0     0.0     0.0  0.0    0.0  0.181818    0.0  0.105263   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-15          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-16          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-17          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-15            NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-16            NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-17            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,0.002190,0.020735,0.012856,0.003620,0.018986,0.002719,0.024600,0.005282,0.012592,0.011478,...,0.003656,0.009267,0.019763,0.038418,0.023101,0.020861,3.083953e-18,0.002534,0.024593,0.018942
2020-08-16,0.001694,0.018632,0.012328,0.001810,0.016678,0.001360,0.021759,0.004665,0.012115,0.009859,...,0.003568,0.006726,0.021481,0.036813,0.018385,0.022224,1.541976e-18,0.004227,0.020842,0.017682
2020-08-17,0.000887,0.017378,0.011892,0.008994,0.015946,0.000680,0.018614,0.002789,0.010621,0.008438,...,0.002909,0.009266,0.016019,0.033915,0.019047,0.020487,7.709882e-19,0.005591,0.036965,0.013308


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,0.004465,0.025918,0.006943,0.001202,0.021071,9.477423e-38,0.026322,0.004581,0.035310,0.002380,...,0.000206,0.006765,0.022759,0.036192,0.093125,0.013223,0.0,0.001058,0.021116,0.025414
2020-08-16,0.004057,0.019626,0.007148,0.000601,0.022164,4.738712e-38,0.019015,0.002902,0.049221,0.001190,...,0.000163,0.003383,0.026935,0.027154,0.046562,0.015871,0.0,0.002423,0.010558,0.020399
2020-08-17,0.002029,0.014199,0.006859,0.000300,0.011082,2.369356e-38,0.019239,0.005119,0.044800,0.001282,...,0.000118,0.028007,0.022088,0.026033,0.044114,0.021572,0.0,0.005928,0.012971,0.021563


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-15,0.009617,0.017314,0.013023,0.003120,0.041636,0.023048,0.052250,0.009502,0.017492,0.006270,...,0.003130,0.008453,0.047721,0.022055,0.021851,0.020723,1.807004e-20,0.021007,0.044640,0.050563
2020-08-16,0.004809,0.015064,0.013511,0.001560,0.020818,0.041644,0.040722,0.006201,0.025250,0.004444,...,0.001565,0.006739,0.051906,0.032939,0.020993,0.034328,9.035018e-21,0.010503,0.043832,0.036273
2020-08-17,0.002404,0.010431,0.012863,0.004256,0.013594,0.020822,0.034881,0.004127,0.028708,0.004253,...,0.002128,0.005453,0.025953,0.035742,0.022558,0.020620,4.517509e-21,0.021046,0.031605,0.437831


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-15,0.009985,0.021811,NaN,0.005113,0.002368,0.013890,0.006371,0.000935,0.009681,5.962972e-47,...,0.012239,0.013820,0.008681,0.006593,0.047045,0.009697,0.011032,0.018919,0.01359,0.014085
2020-08-16,0.008957,0.023297,NaN,0.004848,0.007656,0.011702,0.006385,0.000468,0.006518,2.981486e-47,...,0.013562,0.014999,0.007913,0.005285,0.028291,0.009279,0.009822,0.015664,0.01206,0.016184
2020-08-17,0.007111,0.017518,NaN,0.003633,0.007739,0.010292,0.005017,0.003869,0.005842,1.490743e-47,...,0.010655,0.012645,0.006550,0.006603,0.026966,0.008079,0.006838,0.012151,0.00949,0.014939


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-15      0.006895  0.012161  0.010041  0.011748  0.010280  0.004407   
2020-08-16      0.008422  0.011030  0.010037  0.005874  0.008653  0.003214   
2020-08-17      0.004622  0.006934  0.005846  0.008267  0.006071  0.001607   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-15      0.007887  0.014280  0.002636  0.019801  ...  0.013055   
2020-08-16      0.006482  0.011344  0.001318  0.014917  ...  0.006527   
2020-08-17      0.005135  0.010101  0.001244  0.014081  ...  0.010208   

Province_State                                                              \
Admin2           Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-15      0.02500  0.034748  0.001616   0.012365  0.008899 -0.002609   
2020-08-16      0.01250  0.029719  0.000808   0.008001  0.009644 -0.001304   
2020-08-17      0.00625  0.032932  0.000404   0.007623  0.006107  0.001159   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-15           -1.0  0.044809  0.197917  
2020-08-16           -1.0  0.022405  0.098958  
2020-08-17           -1.0  0.026666  0.120908  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-15,0.004184,0.021086,NaN,0.013825,0.017307,0.012726,0.003662,0.000265,0.000348,NaN,...,0.014113,0.022953,0.011054,0.000017,0.000123,0.004153,0.007548,0.020859,0.009567,0.005504
2020-08-16,0.002620,0.010543,NaN,0.008471,0.007820,0.007477,0.001831,0.000133,0.000174,NaN,...,0.014863,0.017710,0.005527,0.000009,0.000061,0.002076,0.008021,0.010430,0.005265,0.002752
2020-08-17,0.008423,0.005271,NaN,0.004235,0.007249,0.006096,0.000916,0.000403,0.000087,NaN,...,0.015118,0.011308,0.004141,0.000004,0.000031,0.001878,0.005133,0.005215,0.002633,0.001376


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-15      0.001513  0.005505  0.006252  0.001170  0.009122   
2020-08-16      0.000756  0.002753  0.003126  0.000585  0.004561   
2020-08-17      0.000378  0.001376  0.001563  0.000292  0.002281   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-08-15      2.980245e-09  0.000112  0.009386  3.726575e-08  0.002232  ...   
2020-08-16      1.490122e-09  0.000056  0.004693  1.863288e-08  0.001116  ...   
2020-08-17      9.090909e-02  0.000028  0.054978  9.316438e-09  0.000558  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-15         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-16         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-17         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-15       0.005702      NaN    NaN  
2020-08-16       0.002851      NaN    NaN  
2020-08-17       0.001425      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200817,AK,5110,300538,NaN,33.0,NaN,NaN,NaN,4.0,...,305648,0,0,331c3a0940f1eb0806ee76812eaeb05a5ae18b4d,0,0,0,0,0,NaN
1,20200817,AL,109004,743919,NaN,1301.0,12958.0,NaN,1319.0,NaN,...,852923,27,351,fd3b0e913f7262f2ff10651f44973eae3f86a7ff,0,0,0,0,0,NaN
2,20200817,AR,53077,569385,NaN,486.0,3621.0,NaN,NaN,120.0,...,622462,4,51,2eec0948cf075f858c05e90f50e840be2027059b,0,0,0,0,0,NaN
3,20200817,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,cef514690fc398e2a8ac580704942c915fcac050,0,0,0,0,0,NaN
4,20200817,AZ,194005,904383,NaN,1182.0,20756.0,430.0,NaN,266.0,...,1098388,0,1,403fdb4d4f19533718c99f1b9fa31c873371854a,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200817,AK,5110.0,300538.0,NaN,33.0,NaN,NaN,NaN,4.0,...,305648,0,0,331c3a0940f1eb0806ee76812eaeb05a5ae18b4d,0,0,0,0,0,NaN
1,20200817,AL,109004.0,743919.0,NaN,1301.0,12958.0,NaN,1319.0,NaN,...,852923,27,351,fd3b0e913f7262f2ff10651f44973eae3f86a7ff,0,0,0,0,0,NaN
2,20200817,AR,53077.0,569385.0,NaN,486.0,3621.0,NaN,NaN,120.0,...,622462,4,51,2eec0948cf075f858c05e90f50e840be2027059b,0,0,0,0,0,NaN
3,20200817,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,cef514690fc398e2a8ac580704942c915fcac050,0,0,0,0,0,NaN
4,20200817,AZ,194005.0,904383.0,NaN,1182.0,20756.0,430.0,NaN,266.0,...,1098388,0,1,403fdb4d4f19533718c99f1b9fa31c873371854a,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-17,AK,5110,300538,NaN,33.0,NaN,NaN,NaN,4.0,NaN,...,305648,0,0,331c3a0940f1eb0806ee76812eaeb05a5ae18b4d,0,0,0,0,0,NaN
2020-08-17,AL,109004,743919,NaN,1301.0,12958.0,NaN,1319.0,NaN,720.0,...,852923,27,351,fd3b0e913f7262f2ff10651f44973eae3f86a7ff,0,0,0,0,0,NaN
2020-08-17,AR,53077,569385,NaN,486.0,3621.0,NaN,NaN,120.0,472.0,...,622462,4,51,2eec0948cf075f858c05e90f50e840be2027059b,0,0,0,0,0,NaN
2020-08-17,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,cef514690fc398e2a8ac580704942c915fcac050,0,0,0,0,0,NaN
2020-08-17,AZ,194005,904383,NaN,1182.0,20756.0,430.0,NaN,266.0,NaN,...,1098388,0,1,403fdb4d4f19533718c99f1b9fa31c873371854a,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-17,AK,5110.0,300538.0,NaN,33.0,NaN,NaN,NaN,4.0,NaN,...,305648,0,0,331c3a0940f1eb0806ee76812eaeb05a5ae18b4d,0,0,0,0,0,NaN
2020-08-17,AL,109004.0,743919.0,NaN,1301.0,12958.0,NaN,1319.0,NaN,720.0,...,852923,27,351,fd3b0e913f7262f2ff10651f44973eae3f86a7ff,0,0,0,0,0,NaN
2020-08-17,AR,53077.0,569385.0,NaN,486.0,3621.0,NaN,NaN,120.0,472.0,...,622462,4,51,2eec0948cf075f858c05e90f50e840be2027059b,0,0,0,0,0,NaN
2020-08-17,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,cef514690fc398e2a8ac580704942c915fcac050,0,0,0,0,0,NaN
2020-08-17,AZ,194005.0,904383.0,NaN,1182.0,20756.0,430.0,NaN,266.0,NaN,...,1098388,0,1,403fdb4d4f19533718c99f1b9fa31c873371854a,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE